<a href="https://colab.research.google.com/github/ganasg/Colab-notebooks/blob/main/Copy_of_%F0%9F%97%A3%EF%B8%8FQuestionMyDoc%F0%9F%93%84_with_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### The Basics of LangChain

In this notebook we'll explore exactly what LangChain is doing - and implement a straightforward example that lets us ask questions of a document!

First things first, let's get our dependencies all set!

In [ ]:
!pip install openai langchain -q

You'll need to have an OpenAI API key for this next part - see [this](https://www.onmsft.com/how-to/how-to-get-an-openai-api-key/) if you haven't already set one up!

In [ ]:
import os
import getpass

openai_api_key = getpass.getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

OpenAI API Key: ··········


#### Helper Functions (run this cell)

In [ ]:
from IPython.display import display, Markdown

def disp_markdown(text: str) -> None:
  display(Markdown(text))

### Our First LangChain ChatModel



---


<div class="warn">Note: Information on OpenAI's <a href=https://openai.com/pricing>pricing</a> and <a href=https://openai.com/policies/usage-policies>usage policies.</a></div>



---



Now that we're set-up with OpenAI's API - we can begin making our first ChatModel!

There's a few important things to consider when we're using LangChain's ChatModel that are outlined [here](https://python.langchain.com/en/latest/modules/models/chat.html)

Let's begin by initializing the model with OpenAI's `gpt-3.5-turbo` (ChatGPT) model.

We're not going to be leveraging the [streaming](https://python.langchain.com/en/latest/modules/models/chat/examples/streaming.html) capabilities in this Notebook - just the basics to get us started!

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

chat_model = ChatOpenAI(model_name="gpt-3.5-turbo")

If we look at the [Chat completions](https://platform.openai.com/docs/guides/chat) documentation for OpenAI's chat models - we'll see that there are a few specific fields we'll need to concern ourselves with:

`role`
- This refers to one of three "roles" that interact with the model in specific ways.
- The `system` role is an optional role that can be used to guide the model toward a specific task. Examples of `system` messages might be:
  - You are an expert in Python, please answer questions as though we were in a peer coding session.
  - You are the world's leading expert in stamps.

  These messages help us "prime" the model to be more aligned with our desired task!

- The `user` role represents, well, the user!
- The `assistant` role lets us act in the place of the model's outputs. We can (and will) leverage this for some few-shot prompt engineering!

Each of these roles has a class in LangChain to make it nice and easy for us to use!

Let's look at an example.

In [ ]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

# The SystemMessage is associated with the system role
system_message = SystemMessage(content="You are a food critic.")

# The HumanMessage is associated with the user role
user_message = HumanMessage(content="Do you think Kraft Dinner constitues fine dining?")

# The AIMessage is associated with the assistant role
assistant_message = AIMessage(content="Egads! No, it most certainly does not!")

Now that we have those messages set-up, let's send them to `gpt-3.5-turbo` with a new user message and see how it does!

It's easy enough to do this - the ChatOpenAI model accepts a list of inputs!

In [ ]:
second_user_message = HumanMessage(content="What about Red Lobster, surely that is fine dining!")

# create the list of prompts
list_of_prompts = [
    system_message,
    user_message,
    assistant_message,
    second_user_message
]

# we can just call our chat_model on the list of prompts!
chat_model(list_of_prompts)

AIMessage(content="Ah, Red Lobster. While it may have its charm and appeal, I must say that it does not quite reach the level of fine dining. It falls more into the category of casual dining or even chain restaurant fare. However, if you have a hankering for seafood in a relaxed setting, Red Lobster can certainly hit the spot. Just don't expect the same level of refinement and culinary excellence you would find at a true fine dining establishment.", additional_kwargs={}, example=False)

Great! That's inline with what we expected to see!

### PromptTemplates

Next stop, we'll discuss a few templates. This allows us to easily interact with our model by not having to redo work we've already completed!

In [ ]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

# we can signify variables we want access to by wrapping them in {}
system_prompt_template = "You are an expert in {SUBJECT}, and you're currently feeling {MOOD}"
system_prompt_template = SystemMessagePromptTemplate.from_template(system_prompt_template)

user_prompt_template = "{CONTENT}"
user_prompt_template = HumanMessagePromptTemplate.from_template(user_prompt_template)

# put them together into a ChatPromptTemplate
chat_prompt = ChatPromptTemplate.from_messages([system_prompt_template, user_prompt_template])

Now that we have our `chat_prompt` set-up with the templates - let's see how we can easily format them with our content!

NOTE: `disp_markdown` is just a helper function to display the formatted markdown response.

In [ ]:
# note the method `to_messages()`, that's what converts our formatted prompt into
formatted_chat_prompt = chat_prompt.format_prompt(SUBJECT="cheeses", MOOD="quite tired", CONTENT="Hi, what are the finest cheeses?").to_messages()

disp_markdown(chat_model(formatted_chat_prompt).content)

Hello! As an expert in cheeses, I can assure you that there are numerous exceptional cheeses from around the world. However, since I'm currently feeling quite tired, I apologize for not being able to provide an extensive list. Nonetheless, here are a few renowned cheeses that are widely regarded as some of the finest:

1. Parmigiano Reggiano: A hard, granular cheese with a rich and nutty flavor, originating from Italy.

2. Gruyère: A Swiss cheese with a distinctively nutty and creamy taste, often used in dishes like fondue.

3. Roquefort: A French blue cheese made from sheep's milk, known for its tangy and creamy texture.

4. Brie de Meaux: A soft, creamy French cheese with a bloomy rind, offering a delicate and buttery flavor.

5. Manchego: A Spanish sheep's milk cheese with a firm texture and a slightly salty and nutty taste.

6. Gorgonzola: An Italian blue cheese with a creamy, crumbly texture and a sharp, tangy flavor.

7. Stilton: A traditional English blue cheese, creamy and crumbly, with a rich and complex taste.

Remember, this is just a small selection, and there are countless other extraordinary cheeses waiting to be discovered.

### Putting the Chain in LangChain

In essense, a chain is exactly as it sounds - it helps us chain actions together.

Let's take a look at an example.

In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(llm=chat_model, prompt=chat_prompt)

disp_markdown(chain.run(SUBJECT="classic cars", MOOD="angry", CONTENT="Is the 67 Chevrolet Impala a good vehicle?"))

No, the 67 Chevrolet Impala is not a good vehicle. It is a classic masterpiece that deserves to be treated with respect and admiration. It is a symbol of American automotive excellence and craftsmanship. Calling it just a "good vehicle" is an insult to its legacy and heritage. It is infuriating to see such a lack of appreciation for this iconic car.

### Index Local Files

Now that we've got our first chain running, let's talk about indexing and what we can do with it!

For the purposes of this tutorial, we'll be using the word "index" to refer to a collection of documents organized in a way that is easy for LangChain to access them as a "Retriever".

Let's check out the Retriever set-up! First, a new dependency!

In [ ]:
!pip install chromadb tiktoken nltk -q

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Before we can get started with our chain - we'll have to include some kind of text that we want to include as potential context.

Let's use Douglas Adam's [The Hitch Hiker's Guide to the Galaxy](https://erki.lap.ee/failid/raamatud/guide1.txt) as our text file.

In [ ]:
%pwd

'/content'

In [ ]:
!wget https://erki.lap.ee/failid/raamatud/guide1.txt

--2023-09-05 01:50:05--  https://erki.lap.ee/failid/raamatud/guide1.txt
Resolving erki.lap.ee (erki.lap.ee)... 185.158.177.102
Connecting to erki.lap.ee (erki.lap.ee)|185.158.177.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 291862 (285K) [text/plain]
Saving to: ‘guide1.txt.1’

guide1.txt.1        100%[===================>] 285.02K   548KB/s    in 0.5s    

2023-09-05 01:50:08 (548 KB/s) - ‘guide1.txt.1’ saved [291862/291862]



In [ ]:
from langchain.document_loaders import TextLoader
loader = TextLoader('guide1.txt', encoding='utf8')

Now we can set up our first Index!

More detail can be found [here](https://python.langchain.com/en/latest/modules/indexes/getting_started.html) but we'll skip to a more functional implementation!

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
index = VectorstoreIndexCreator().from_loaders([loader])

Now that we have our Index set-up, we can query it straight away!

In [ ]:
query = "What are the importances of towels?"
index.query_with_sources(query)

{'question': 'What are the importances of towels?',
 'answer': ' A towel is a massively useful item for an interstellar hitch hiker, as it can be used for warmth, protection, and signaling for help. It also has immense psychological value, as it can make a strag (non-hitch hiker) assume that the hitch hiker is well-prepared and capable.\n',
 'sources': 'guide1.txt'}

### Putting it All Together

Now that we have a simple idea of how we prompt, what a chain is, and has some local data - let's put it all together!

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import NLTKTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.indexes.vectorstore import VectorstoreIndexCreator

with open("guide1.txt") as f:
    hitchhikersguide = f.read()

text_splitter = NLTKTextSplitter()
texts = text_splitter.split_text(hitchhikersguide)

embeddings = OpenAIEmbeddings()

In [ ]:
docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))]).as_retriever()

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

chain = load_qa_chain(OpenAI(temperature=0), chain_type="refine")
query = "What is the wind speed velocity of a swallow?"
docs = docsearch.get_relevant_documents(query)
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': '\n\nThe wind speed velocity of a swallow is not mentioned in the context information.'}

This notebook was authored by [Chris Alexiuk](https://www.linkedin.com/in/csalexiuk/)